# HTTP wrapper for kachaka-api

kachaka-apiライブラリをHTTP経由でアクセスできるようにするデモです。  
応答はJSONで返されます。

### GET使用例

引数の無い関数はGETで呼ぶことができます。

```
curl <カチャカのIPアドレス>:26502/kachaka/get_robot_serial_number
curl <カチャカのIPアドレス>:26502/kachaka/get_auto_homing_enabled
curl <カチャカのIPアドレス>:26502/kachaka/return_home
```

### POST使用例:

POSTで引数をJSON形式で与えることで、引数のある関数を実行することができます。なお、curlは `--json` オプションを渡すことで自動的にPOSTになります。

```
curl <カチャカのIPアドレス>:26502/kachaka/set_auto_homing_enabled --json '{"enable": false}'
curl <カチャカのIPアドレス>:26502/kachaka/move_shelf --json '{"shelf_name_or_id":"シェルフ", "location_name_or_id":"キッチン"}'
curl <カチャカのIPアドレス>:26502/kachaka/speak --json '{"text":"APIから指令を受信しました"}'
```

In [ ]:
import io

import kachaka_api
from fastapi import FastAPI, HTTPException
from fastapi.responses import StreamingResponse
from google._upb._message import RepeatedCompositeContainer
from google.protobuf.json_format import MessageToDict

app = FastAPI()
kachaka_client = kachaka_api.aio.KachakaApiClient()


@app.on_event("startup")
async def init_channel() -> None:
    await kachaka_client.update_resolver()


def to_dict(response):
    if response.__class__.__module__ == "kachaka_api_pb2":
        return MessageToDict(response)
    if (
        isinstance(response, tuple)
        or isinstance(response, list)
        or isinstance(response, RepeatedCompositeContainer)
    ):
        return [to_dict(r) for r in response]
    return response


async def run_method_or_404(attr: str, args: dict = {}):
    if not hasattr(kachaka_client, attr):
        raise HTTPException(status_code=404, detail="Method not found")
    method = getattr(kachaka_client, attr)
    response = await method(**args)
    return to_dict(response)


@app.get("/kachaka/{front_or_back}_camera_image.jpeg")
async def front_or_back_camera_image(front_or_back: str):
    if front_or_back == "front":
        response = await kachaka_client.get_front_camera_ros_compressed_image()
    elif front_or_back == "back":
        response = await kachaka_client.get_back_camera_ros_compressed_image()
    else:
        raise HTTPException(status_code=404, detail="Camera not found")
    image_bytes = io.BytesIO(response.data)
    return StreamingResponse(image_bytes, media_type="image/jpeg")


@app.get("/kachaka/{method:path}")
async def get(method: str):
    return await run_method_or_404(method, {})


@app.post("/kachaka/{method:path}")
async def post(method: str, params: dict):
    return await run_method_or_404(method, params)

In [ ]:
import uvicorn

config = uvicorn.Config(app)
config.host = "0.0.0.0"
config.port = 26502
server = uvicorn.Server(config)
await server.serve()